In [1]:
import gymnasium as gym
import ale_py
import torch
from torch import nn

from math import prod

/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/shimmy/registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/shimmy/registration.py", line 205, in _register_atari_envs
    import ale_py
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/ale_py/__init__.py", line 66, in <module>
    register_v0_v4_envs()
  File "/home/fitti/.conda/envs/puffer/lib/python3.11/site-packages/ale_py/registration.py", line 176, in register_v0_v4_envs
    _register_rom_configs(legacy_games, obs_types, versions)
  File "/home/fitti/.conda

In [2]:
env = gym.make("ALE/Breakout-v5")

A.L.E: Arcade Learning Environment (version 0.9.0+750d7f9)
[Powered by Stella]


In [3]:
ob_shape, n_actions = env.observation_space.shape, env.action_space.n

In [4]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, first_channels, second_channels, out_channels):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, first_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(first_channels, second_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(second_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

    def forward(self, x):
        return self.encoder(x)

In [5]:
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, first_channels, second_channels, out_channels, out_size=None):
        super().__init__()
        if out_size:
            upsampler = nn.UpsamplingBilinear2d(size=out_size)
        else:
            upsampler = nn.UpsamplingBilinear2d(scale_factor=2)
        self.decoder = nn.Sequential(
            upsampler,
            nn.Conv2d(in_channels, first_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(first_channels, second_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(second_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.decoder(x)

In [6]:
class Encoder(nn.Module):
    def __init__(self, ob_shape):
        super().__init__()
        c, h, w = ob_shape
        self.encoder = nn.Sequential(
            EncoderBlock(c, 16, 32, 32),
            EncoderBlock(32, 48, 48, 64),
            EncoderBlock(64, 128, 128, 64),
        )

    def forward(self, x):
        return self.encoder(x)

In [7]:
class Decoder(nn.Module):
    def __init__(self, ob_shape):
        super().__init__()
        c, h, w = ob_shape
        self.decoder = nn.Sequential(
            DecoderBlock(64, 128, 128, 64),
            DecoderBlock(64, 48, 48, 32),
            DecoderBlock(32, 32, 16, c, out_size=[h, w]),
        )

    def forward(self, x):
        return self.decoder(x)

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, in_shape, n_hiddens, n_actions):
        super().__init__()
        self.input = nn.Sequential(
            nn.Flatten(),
            nn.Linear(prod(in_shape), n_hiddens),
        )
        self.step = nn.Linear(n_hiddens, n_hiddens)
        self.out = nn.Linear(n_hiddens, n_actions)
        
    def forward(self, x, state):
        state = self.step(state) + self.input(x)
        out = self.out(state)

        return out, state